# Piping Chains and the RunnablePassthrough Class

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
%pip install -qU langchain-openai

In [3]:
pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: 


In [ ]:
# %load_ext dotenv
# %dotenv

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [7]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [8]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [9]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if (await self.run_code(code, result,  async_=asy)):


In [10]:
string_parser = StrOutputParser()

In [11]:
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

In [12]:
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. R Programming Language\n3. SQL (Structured Query Language)\n4. Tableau\n5. Jupyter Notebook'}


In [13]:
print(chain_strategy.invoke({'tools':'''
1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop
'''}))

1. Python: Start with the basics of Python, such as variables, data types, operators, and control flow. Once you have a good understanding of these, move on to more complex topics like functions, classes, and error handling. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding regularly on platforms like LeetCode or HackerRank. Work on small projects to apply what you've learned.

2. R Programming: Start with the basics of


In [14]:
chain_combined = chain_tools | chain_strategy

In [15]:
print(chain_combined.invoke({'job title':'data scientist'}))

1. Python: Start with the basics of Python, such as variables, data types, loops, and functions. Use online resources like Codecademy, Coursera, or edX for structured learning. Practice coding problems on platforms like LeetCode or HackerRank. Work on small projects, such as a simple calculator or a text-based game, to apply what you've learned. As you get more comfortable, learn more advanced topics like object-oriented programming, data structures, and algorithms.

2.


In [16]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} |
              chat_template_strategy | chat | string_parser)